# Introduction
This notebook is about brain tumor classification using two deep learning methods: MLP and CNN (ResNet50). Models are trained with multiclass labels, however, you can do both binary classification and multiclassification using this code.

The code is structured as follows:
1. Load packages
2. Preprocess data
3. Train and test MLP models
4. Train and test ResNet50 models

You can run part 1 and part 2 first, then run either part 3 or part 4 to train and test your own model.

ATTENTION: This code is based on Windows. If you want to run it on Google Colab or other linux based servers, please change all '\\'  to '/'.

## 1. Load packages
The model is built using tensorflow 2.

In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd
import cv2

from sklearn import metrics
from sklearn.model_selection import train_test_split

## 2. Preprocess data
This part includes:

1. labels encoding
2. train set and test set preparation
3. the function used to choose predict labels
4. the function used to convert multiclass labels to binary labels

If you want to do multiclassification, just run 1-3, if you want to do binary classification, run all the steps.

In [2]:
data_path='.\\dataset\\'

In [3]:
#label encoding
slabels=pd.read_csv(data_path+'label.csv')['label']
labels=[]
for s in slabels:
    if s=='no_tumor':
        labels.append(0)
    elif s=='meningioma_tumor':
        labels.append(1)
    elif s=='glioma_tumor':
        labels.append(2)
    else:
        labels.append(3)
labels=np.array(labels)

In [4]:
#choose predict label
def predict(predicts):
    predict_labels=[]
    for p in predicts:
        p=np.argmax(p)
        predict_labels.append(p)  
    return np.array(predict_labels)

In [5]:
#multiclass to binary
def multi_to_binary(labels):
    binary=[]
    for label in labels:
        binary.append(0) if label=='no_tumor' else binary.append(1)
    return binary

## 3.Train and test MLP models
Through part you can:

1. prepare trainset and testset
2. load my trained MLP model
3. train your own MLP model
4. test the model for multiclass and binary tasks

### 3.1 Prepare trainset and testset

In [6]:
#image processing
names=pd.read_csv(data_path+'label.csv')['file_name']
imgs=[]
for name in names:
    img=cv2.imread(data_path+'image\\'+name,cv2.IMREAD_GRAYSCALE)
    imgs.append(np.array(img))
imgs=np.array(imgs)

In [7]:
#split trainset and testset
train_examples,test_examples,train_labels,test_labels=train_test_split(imgs,labels,test_size=0.2,random_state=3)

### 3.2 Load trained model

In [8]:
#load the trained model
mlp=tf.keras.models.load_model('.\\models\\mlp.h5')

### 3.3 Train your own MLP

In [ ]:
mlp = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[512, 512]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(4, activation="softmax")
])

In [ ]:
mlp.compile(loss = "sparse_categorical_crossentropy",
             optimizer = "adam", 
             metrics = ["accuracy"])

In [7]:
mlp.fit(train_examples, train_labels,epochs=20)

Train on 2400 samples
Epoch 1/20
2400/2400 [==============================] - 67s 28ms/sample - loss: 4328.8874 - accuracy: 0.4304
Epoch 2/20
2400/2400 [==============================] - 62s 26ms/sample - loss: 804.7284 - accuracy: 0.5558
Epoch 3/20
2400/2400 [==============================] - 67s 28ms/sample - loss: 457.4655 - accuracy: 0.6208
Epoch 4/20
2400/2400 [==============================] - 66s 27ms/sample - loss: 202.2375 - accuracy: 0.7387
Epoch 5/20
2400/2400 [==============================] - 61s 25ms/sample - loss: 179.6432 - accuracy: 0.7275
Epoch 6/20
2400/2400 [==============================] - 59s 25ms/sample - loss: 129.7456 - accuracy: 0.7487
Epoch 7/20
2400/2400 [==============================] - 59s 25ms/sample - loss: 103.7183 - accuracy: 0.7796
Epoch 8/20
2400/2400 [==============================] - 60s 25ms/sample - loss: 62.0880 - accuracy: 0.8188
Epoch 9/20
2400/2400 [==============================] - 60s 25ms/sample - loss: 72.0776 - accuracy: 0.7954
Epoch 1

### 3.4 Test the model

In [9]:
#prediction
predicts=mlp.predict(test_examples)
predict_labels=predict(predicts) 

In [10]:
#multiclass accuracy
score=metrics.accuracy_score(test_labels,predict_labels)
score

0.7716666666666666

In [11]:
#binary accuracy
predict_binary=multi_to_binary(predict_labels)
test_binary=multi_to_binary(test_labels)
score_binary=metrics.accuracy_score(test_binary,predict_binary)
score_binary

1.0

## 4.Train and test ResNet50 models
Through part you can:

1. prepare trainset and testset
2. load my trained ResNet50 model
3. train your own ResNet model
4. test the model for multiclass and binary tasks

### 4.1 Prepare trainset and testset

In [12]:
#image processing
names=pd.read_csv(data_path+'label.csv')['file_name']
imgs=[]
for name in names:
    img=cv2.imread(data_path+'image\\'+name)
    img=cv2.resize(img,(224,224),interpolation=cv2.INTER_CUBIC)
    imgs.append(np.array(img))
imgs=np.array(imgs)

In [13]:
#split trainset and testset
train_examples,test_examples,train_labels,test_labels=train_test_split(imgs,labels,test_size=0.2,random_state=3)

### 4.2 Load the trained model

In [14]:
res=tf.keras.models.load_model('.\\models\\ResNet.h5')

### 4.3 Train your own ResNet50

In [ ]:
res=tf.keras.applications.resnet50.ResNet50(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=None, pooling=None, classes=1000,
)

In [ ]:
res.compile(loss = "sparse_categorical_crossentropy",
             optimizer = "adam", 
             metrics = ["accuracy"])

In [ ]:
res.fit(train_examples, train_labels,epochs=20)

### 4.4 Test the model

In [15]:
#prediction
predicts=res.predict(test_examples)
predict_labels=predict(predicts) 

In [16]:
#multiclass accuracy
score=metrics.accuracy_score(test_labels,predict_labels)
score

0.96

In [17]:
#binary accuracy
predict_binary=multi_to_binary(predict_labels)
test_binary=multi_to_binary(test_labels)
score_binary=metrics.accuracy_score(test_binary,predict_binary)
score_binary

1.0